# Sabit işlemler

## İmportlar

In [ ]:
import torch
import os
from torch.utils.data import DataLoader, Dataset, random_split
from torch.optim import AdamW
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from tqdm import tqdm  # İlerleme çubuğu için tqdm kütüphanesini kullanıyoruz.

## Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Donanım seç

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## Sabitler

In [ ]:
num_labels = 2

lr = 6e-4
bs = 16

base_path = "/content/drive/My Drive/yl_tez"
epoch_sayisi = 20
max_len = 128
# anahatlar
metin = "text"
label = "diabetes"
model_name = "ytu-ce-cosmos/turkish-gpt2-medium"
veri_seti_adi = "ana_veri_seti.csv"
#veri_seti_adi =  "ana_veri_seti_deneme.csv"
# Veri seti dosya yolu
data_filepath = os.path.join(base_path, veri_seti_adi)
# Dosya adları
model_adi = "BestModelParameters"
checkpoint_path_name = "checkpoint"
checkpoint_name = 'checkpoint.pth'
best_checkpoint_name = 'best_checkpoint.pth'
checkpoint_pth = os.path.join(base_path, checkpoint_path_name)
model_yolu = os.path.join(base_path, model_adi)
model_path = model_name.replace("/", "_").replace("-", "_")
model_path_without_label = os.path.join(base_path, model_path)
gorsel_klasor_adi = "gorseller"
gorsel_yolu = os.path.join(base_path, gorsel_klasor_adi)
SORU_TOKEN ="<|soru|>"
CEVAP_TOKEN ="<|cevap|>"

# Veri setini oku

In [ ]:
# Veri dosyasını oku
df = pd.read_csv(data_filepath)

In [ ]:
print(df.head().to_markdown())
len(df)

|    | text                                                                                                                                                                                                                                                                                                                                                                                                     |
|---:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Soru: hba1c verisine göre Hastanın HbA1c değeri, diyabet riskini gösteriyor Senior yaş grubundaki birey Hastanın hipertansiyonu tespit edilmemiştir Tetkikler, hastanın kalp rahatsızlığı old

100569

# Fonksiyonlar

## Veri Seti Oluşturma Fonksiyonu

In [ ]:
class QADataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.decoded_texts = []
        # Tokenizer için bir padding token'ı atayın, eğer yoksa
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.questions = self.data['text'].apply(lambda  x: SORU_TOKEN + " " + preprocess_text(x.split("Cevap:")[0].strip().replace("Soru:","")) + " ")
        self.answers = self.data['text'].apply(lambda  x: CEVAP_TOKEN + " " + preprocess_text(x.split("Cevap:")[1].strip()) + " ")

        self.inputs = []
        self.attention_masks = []

        for question, answer in zip(self.questions, self.answers):
            # Soru ve cevabı tokenizer ile encode edin
            encoded_pair = self.tokenizer.encode_plus(question, answer,
                                                      add_special_tokens=True,
                                                      max_length=max_len,
                                                      padding='max_length',
                                                      truncation=True,
                                                      return_tensors="pt")
            # encodeanan değer nasıl decodelanıyor desti için
            #decoded_text = self.tokenizer.decode(encoded_pair['input_ids'][0], skip_special_tokens=False)
            #self.decoded_texts.append(decoded_text)
            self.inputs.append(encoded_pair['input_ids'])
            self.attention_masks.append(encoded_pair['attention_mask'])

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx].squeeze(),  # Batch boyutunu kaldır
            'attention_mask': self.attention_masks[idx].squeeze()  # Batch boyutunu kaldır
        }


## Metrik Hesaplama Fonksiyonu

In [ ]:
def calculate_perplexity(loss):
    return np.exp(loss)

# Özel collate_fn fonksiyonu
def collate_fn(batch):
    # Tokenizer'ın pad_token_id'sini kullanarak padding yapın
    input_ids = [item["input_ids"].squeeze(0) for item in batch]  # squeeze(0) boyutunu kaldırır
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)

    # Dikkat maskesi oluşturun (padding olan yerlerde 0, diğer yerlerde 1)
    attention_mask = torch.zeros_like(input_ids_padded)
    attention_mask[input_ids_padded != tokenizer.pad_token_id] = 1

    return {"input_ids": input_ids_padded, "attention_mask": attention_mask}

## Metrik Çizdirme Fonksiyonu

In [ ]:
def plot_metrics(losses, perplexities, lr, bs, epochs, gorsel_yolu = gorsel_yolu, gorsel_adi = "Normal"):
    os.makedirs(gorsel_yolu, exist_ok=True)  # Klasör yoksa
    # Kayıp grafiğini kaydetme yolu
    loss_graph_path = os.path.join(gorsel_yolu, f"{gorsel_adi}_lr_{lr}_bs_{bs}_loss.png")
    # Perplexity grafiğini kaydetme yolu
    perplexity_graph_path = os.path.join(gorsel_yolu, f"{gorsel_adi}_lr_{lr}_bs_{bs}_perplexity.png")

    epochs_range = range(1, epochs + 1)

    # Kayıp grafiği
    plt.figure(figsize=(6, 4))
    plt.plot(epochs_range, losses, label='Loss')
    plt.title(f'{gorsel_adi} Loss with lr={lr}, bs={bs}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.savefig(loss_graph_path)
    plt.close()  # Aktif figürü kapat

    # Perplexity grafiği
    plt.figure(figsize=(6, 4))
    plt.plot(epochs_range, perplexities, label='Perplexity')
    plt.title(f'{gorsel_adi} Perplexity with lr={lr}, bs={bs}')
    plt.xlabel('Epoch')
    plt.ylabel('Perplexity')
    plt.legend()
    plt.tight_layout()
    plt.savefig(perplexity_graph_path)
    plt.close()  # Aktif figürü kapat

In [ ]:
def find_unique_words(texts):
    # Tüm metinlerdeki kelimeleri bulun ve sayın
    words = Counter(re.findall(r'\w+', ' '.join(texts)))
    return list(words.keys())


## Kilometre Taşı Fonksiyonu

In [ ]:
def save_checkpoint(model, optimizer, epoch, loss, losses, perplexities, val_losses, val_perplexities, checkpoint_path, checkpoint_name = checkpoint_name):
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'losses': losses,
        'perplexities': perplexities,
        'val_losses': val_losses,
        'val_perplexities': val_perplexities
    }
    torch.save(checkpoint, os.path.join(checkpoint_path, checkpoint_name))
    print(f"Checkpoint şuraya kaydedildi -> {checkpoint_path} loss değeri: {loss}")
def load_checkpoint(checkpoint_path, model, optimizer,checkpoint_name = checkpoint_name):
    checkpoint_filepath = os.path.join(checkpoint_path, checkpoint_name)
    if os.path.exists(checkpoint_filepath):
        print(f"Checkpoint şu konumdan yükleniyor ->  {checkpoint_filepath}")
        checkpoint = torch.load(checkpoint_filepath, map_location=torch.device(device))
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint.get('loss', float('inf'))
        losses = checkpoint.get('losses', [])
        perplexities = checkpoint.get('perplexities', [])
        val_losses = checkpoint.get('val_losses', [])
        val_perplexities = checkpoint.get('val_perplexities', [])
        return model, optimizer, epoch + 1, loss, losses, perplexities, val_losses, val_perplexities
    else:
        print("Hiçbir checkpoint bulunamadı, sıfırdan başlanıyor.")
        return model, optimizer, 0, float('inf'), [], [], [], []

## Eğitim Fonksiyonu

In [ ]:
def train(device, new_tokens, train_dataloader, validation_dataloader, checkpoint_path = checkpoint_pth):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    if new_tokens:
        tokenizer.add_tokens(new_tokens)
        model.resize_token_embeddings(len(tokenizer))
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=lr)

    # Checkpoint'ten modeli, optimizer'ı, epoch bilgisini, en iyi loss değerini ve metrik listelerini yükle
    model, optimizer, start_epoch, best_loss, losses, perplexities, val_losses, val_perplexities = load_checkpoint(checkpoint_path, model, optimizer)
    if start_epoch == epoch_sayisi:
        print("Eğitim zaten tamam...")
        return None
    best_model, _, _, _, _, _, _, _ = load_checkpoint(checkpoint_path, model, optimizer, best_checkpoint_name)
    for epoch in range(start_epoch, epoch_sayisi):
        print(f"Epoch {epoch} başlıyor...")
        model.train()
        total_loss = 0
        total_perplexity = 0
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch}", unit="batch"):
            optimizer.zero_grad()
            inputs = {key: value.to(device) for key, value in batch.items()}
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            batch_loss = loss.item()
            batch_perplexity = calculate_perplexity(batch_loss)  # Perplexity hesaplama fonksiyonu
            total_loss += batch_loss
            total_perplexity += batch_perplexity

        epoch_loss = total_loss / len(train_dataloader)
        epoch_perplexity = total_perplexity / len(train_dataloader)
        print(f"Epoch {epoch} | Loss: {epoch_loss:.4f} | Perplexity: {epoch_perplexity:.4f}")

        # Doğrulama süreci
        model.eval()  # Modeli değerlendirme moduna al
        validation_loss = 0
        total_validation_perplexity = 0
        with torch.no_grad():
            for batch in validation_dataloader:
                inputs = {key: value.to(device) for key, value in batch.items()}
                outputs = model(**inputs, labels=inputs["input_ids"])
                batch_loss = outputs.loss.item()
                validation_loss += batch_loss

                # Perplexity hesapla (Bu örnekte calculate_perplexity fonksiyonunuzun nasıl tanımlandığına bağlıdır)
                batch_perplexity = calculate_perplexity(batch_loss)
                total_validation_perplexity += batch_perplexity

        validation_loss /= len(validation_dataloader)
        validation_perplexity = total_validation_perplexity / len(validation_dataloader)

        # Doğrulama kaybı ve şaşkınlık metriklerini yazdır
        print(f"Validation Loss: {validation_loss:.4f}")
        print(f"Validation Perplexity: {validation_perplexity:.4f}")

        # Kaydedilen doğrulama metrik listelerini güncelle
        val_losses.append(validation_loss)
        val_perplexities.append(validation_perplexity)

        # Kaydedilen metrik listelerini ve en iyi modeli güncelle
        losses.append(epoch_loss)
        perplexities.append(epoch_perplexity)
        save_checkpoint(model, optimizer, epoch, best_loss, losses, perplexities, val_losses, val_perplexities, checkpoint_path, checkpoint_name)
        is_best = validation_loss < best_loss
        if is_best:
            best_loss = validation_loss
            print(f"Yeni best model loss değeri {best_loss:.4f}. Checkpoint kaydediliyor...")
            save_checkpoint(model, optimizer, epoch, best_loss, losses, perplexities, val_losses, val_perplexities, checkpoint_path, best_checkpoint_name)
            best_model = model
    # Eğitim ve doğrulama metriklerinin grafiklerini çizdir
    plot_metrics(losses, perplexities, lr, len(train_dataloader.dataset), epoch_sayisi)
    plot_metrics(val_losses, val_perplexities, lr, len(validation_dataloader.dataset), epoch_sayisi, gorsel_adi = "val")

    return best_model

# Tokenizer Güncelleme

In [ ]:
texts = df[metin].tolist()
tokenizer = AutoTokenizer.from_pretrained(model_name)


unique_words = find_unique_words(texts)

# Tokenizer'da olmayan kelimeleri tespit et
new_tokens = [word for word in unique_words if tokenizer.convert_tokens_to_ids(word) == tokenizer.unk_token_id]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/585k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

# Veri Seti Okuma

In [ ]:
dataset = QADataset(dataframe=df, tokenizer=tokenizer)

In [ ]:
# Örnek olarak, dataset'inizin %80'inin eğitim, %20'sinin doğrulama için kullanılacağını varsayalım.
train_size = int(0.8 * len(dataset))
validation_size = len(dataset) - train_size

# Dataset'i rastgele eğitim ve doğrulama setlerine böl
train_dataset, validation_dataset = random_split(dataset, [train_size, validation_size])

# DataLoader'ları oluştur
train_dataloader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=bs)

In [ ]:
len(dataset)

100569

# Model Eğitme

In [ ]:
model = train(device, new_tokens, train_dataloader, validation_dataloader)

config.json:   0%|          | 0.00/946 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Checkpoint şu konumdan yükleniyor ->  /content/drive/My Drive/yl_tez/checkpoint/checkpoint.pth
Eğitim zaten tamam...


In [ ]:
if model is not None:
    # Model ve tokenizer kaydetme
    model.save_pretrained(model_yolu)
    dataset.tokenizer.save_pretrained(model_yolu)

# Model Test Etme

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_yolu)
model = AutoModelForCausalLM.from_pretrained(model_yolu)
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50572, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50572, bias=False)
)

In [ ]:
def generate_text(prompt_text, max_length=100):
    # Girdi metnini tokenize et ve attention mask oluştur
    encoded_input = tokenizer.encode(prompt_text, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)

    # Metin üretimi
    output_sequences = model.generate(
        input_ids=encoded_input,
        max_length=max_length + 20,  # Cevap için ekstra uzunluk
        temperature=1.0,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )

    # Üretilen çıktıyı metne dönüştür
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    return generated_text.replace(prompt_text,"")

In [ ]:
while True:
    # Kullanıcıdan soru girdisi alma
    prompt_text = input("Soru: ")
    if prompt_text == "exit":
        break
    # Tam soru metnini oluştur
    full_prompt_text = f"Soru: {prompt_text} Cevap:"

    # Metin üretimi ve çıktının gösterilmesi
    generated_text = generate_text(full_prompt_text)
    print("Modelin ürettiği cevap:", generated_text)

Soru: kalp çarpıntııs var. HbA1c seviyesi düzensiz. elektronik sigara kullanıyor. yaşlı, ve erkek kilosu normal. hipertansiyonu yok.
Modelin ürettiği cevap: etkikler, hastanın diyabet hastası olmadığını göstermektedir Hasta, vücut kitle indeksi düşük seviyede Kan şekeri değerleri sağlıklı görünüyor. Aktif yaş grubundaki hasta Hasta hakkında sigara geçmişi verisi yokturHasta için diyabet tanısı konulmamıştır
Soru: ben erkeğim sigara içiyorum
Modelin ürettiği cevap:  edilmiştir Hastanın vücut kitle indeksi yüksek Hastada herhangi bir kalp rahatsızlığı saptanmamıştır kadındır Yetişkinlik dönemindeki birey hba1c verisine göre Hasta prediyabet durumunda Kişinin kan şekeri seviyesi normal aralıktadır. Sigara kullanımı hakkında herhangi bir bilgi mevcut değildirHastanın diyabet hastası olmadığı belirlenmiştir
Soru: aşırı düşük değerindeki bu çevre, tümörün büyüklüğünü gösteriyor. aşırı yüksek doku özelliğine sahip bir tümör yapısı. Bu tümör, dengeli düzgünlük seviyesine sahip. Bu özelliğiyle 